In [5]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import chrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [31]:
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options , service=Service(ChromeDriverManager().install()))
driver.get('https://www.ad.co.il/nadlanprice')
search = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, 'tbSearchTerm'))
)
search.send_keys("חיפה")
search_box = driver.find_element(By.ID, 'btnSearchFilter')
search_box.send_keys(Keys.RETURN)

In [12]:
try:
    viewContainer = WebDriverWait(driver , 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "viewContainer"))
        print(element.text)
    )
except:
    driver.quit()


SyntaxError: invalid syntax (3222754451.py, line 4)